### Preprocessing

In [1]:
# Import packages
import os
os.chdir("../src/preprocessing") # Change current working directory to load preprocessing functions
#from create_embedding import _load_model, preprocess_folder, apply_embedding_model_to_svgs, combine_embeddings, encode_pkl, encode_svg, decode_z
from create_svg_embedding import _load_model, apply_embedding_model_to_svgs, combine_embeddings, encode_svg, decode_z
from deepsvg.svglib.svg import SVG
from deepsvg import utils
os.chdir("../..")

In [2]:
from src.data.insert_ids import insert_ids_in_folder
from src.data.rename_logos import rename_logos
from src.data.get_svg_meta_data import get_svg_meta_data
from src.data.generate_random_animations import random_animation_vector
from src.data.expand_viewbox import expand_viewbox_in_folder
from src.preprocessing.sort_paths import sort_by_relevance
from src.preprocessing.decompose_logo import decompose_logos_in_folder
from src.features.get_style_attributes import *
#from src.features.get_style_attributes_folder import get_local_style_attributes, get_global_style_attributes, combine_style_attributes
from src.models_output.insert_animation import create_animated_svg
from src.models_output.transform_animation_predictor_output import transform_animation_predictor_output
import pandas as pd
import numpy as np
import importlib, glob, pickle, torch, logging
from datetime import datetime
from concurrent import futures
from tqdm import tqdm

#### 1 Rename and insert IDs in SVGs
Logos are renamed and an ID called animation_id is inserted for every path in a SVG.

In [8]:
expand_viewbox_in_folder('data/initial_svgs', 50, "data/svgs_expanded")

In [3]:
rename_logos("data/svgs_expanded", "data/svgs_expanded_renamed")

In [4]:
insert_ids_in_folder("data/svgs_expanded_renamed", "data/svgs")

#### 2 Decompose SVGs
Decompose SVGs by saving every path in a seperate SVG file. Decomposed SVGs are save to data/decomposed_svgs.

In [6]:
decompose_logos_in_folder("data/svgs")

#### 3 Simplify SVGs
Creates simplified SVGs and its tensors on SVG and path level  
###### UPDATE: NOT NEEDED ANYMORE

In [7]:
preprocess_folder(data_folder="data/svgs")

100%|████████████████████████████████████████████████████████████████████████████████| 192/192 [00:54<00:00,  3.52it/s]


In [8]:
preprocess_folder(data_folder="data/decomposed_svgs")

100%|██████████████████████████████████████████████████████████████████████████████| 5109/5109 [02:42<00:00, 31.53it/s]


#### 4 Get SVGs Metadata

In [4]:
meta_data = get_svg_meta_data("data/svgs")
meta_data

100%|████████████████████████████████████████████████████████████████████████████████| 192/192 [00:06<00:00, 28.10it/s]


,id,total_len,nb_groups,len_groups,max_len_group,start_pos
0,logo_0,86,10,"[7, 7, 5, 4, 4, 7, 14, 16, 7, 15]",16,"[P(11.9639253616333, 4.5934014320373535), P(11..."
1,logo_1,114,8,"[7, 7, 8, 10, 23, 23, 15, 21]",23,"[P(8.111812591552734, 9.714151382446289), P(8...."
2,logo_10,112,13,"[11, 14, 11, 13, 9, 9, 11, 6, 6, 5, 6, 6, 5]",14,"[P(5.903238773345947, 10.45088005065918), P(12..."
3,logo_100,372,30,"[120, 5, 5, 15, 5, 5, 13, 13, 8, 13, 13, 5, 11...",120,"[P(12.465529441833496, 8.196613311767578), P(9..."
4,logo_101,361,35,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 11, 11, 16, 21, 18...",32,"[P(11.995269775390625, 9.817630767822266), P(1..."
...,...,...,...,...,...,...
187,logo_97,294,37,"[5, 3, 17, 5, 5, 5, 8, 6, 17, 13, 12, 12, 14, ...",17,"[P(6.673834323883057, 9.33651351928711), P(8.7..."
188,logo_98,299,38,"[4, 5, 17, 5, 5, 5, 8, 6, 13, 17, 12, 12, 14, ...",17,"[P(6.667652130126953, 8.149038314819336), P(6...."
189,logo_96,463,32,"[4, 27, 5, 5, 17, 19, 14, 19, 33, 13, 24, 19, ...",33,"[P(5.76159143447876, 9.227791786193848), P(4.5..."
190,logo_95,1205,41,"[81, 86, 56, 376, 11, 126, 17, 5, 5, 5, 10, 7,...",376,"[P(17.007122039794922, 7.569143772125244), P(1..."


In [6]:
meta_data.to_csv("data/svgs_metadata.csv", index=False)

In [5]:
output = open("data/svgs_metadata.csv", 'wb')
pickle.dump(meta_data, output)
output.close()

#### 5 Get Path Embeddings
Apply hierarchical generative network to produce latent vector z

In [37]:
#model = "models/20210301_1813_model_batch60_epoch30_decomposed_svgs.pth.tar"
model = "models/hierarchical_ordered.pth.tar"

In [3]:
apply_embedding_model_to_svgs(data_folder="data/decomposed_svgs", save=False)

100%|██████████████████████████████████████████████████████████████████████████████| 5109/5109 [04:05<00:00, 20.84it/s]


,filename,animation_id,0,1,2,3,4,5,6,7,...,246,247,248,249,250,251,252,253,254,255
0,logo_0,1,0.760854,-1.654953,0.728048,-0.192816,-0.039667,0.104971,-1.947641,-1.436980,...,-1.857717,-1.807807,0.185051,-0.410260,0.014543,-1.204671,-1.092592,0.305544,-0.936117,0.379724
1,logo_0,0,0.789936,-0.786410,-0.309709,1.362506,-0.290023,1.017092,-0.394151,-1.450244,...,0.525660,-0.590144,-0.130421,0.159403,-0.080556,-0.729687,-1.473149,0.363847,0.311790,-0.102659
2,logo_0,2,0.307773,-0.866350,1.712584,-0.459635,-0.900046,0.969765,-0.638381,0.001584,...,-1.170392,-1.855947,0.057350,1.037182,0.677306,-1.379392,-0.026326,-0.223816,0.420645,0.522583
3,logo_0,5,-0.450654,-0.920692,0.279545,-0.288098,-0.538055,0.651536,-1.107719,-1.459157,...,-1.789967,-0.815155,1.254517,-0.502050,0.011259,-0.860543,-1.159536,0.572207,-1.400677,0.405144
4,logo_0,3,0.465979,-1.299480,0.643814,-0.791493,-0.280398,0.867627,-1.050468,-1.022961,...,-1.385686,-2.488016,0.345313,1.659617,0.849781,-1.353936,-0.586149,-0.253920,0.436272,0.142771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5077,logo_9,4,0.760010,-1.201990,0.708384,-0.223727,-0.600647,0.471647,-1.613992,-0.362167,...,-2.235480,-3.380206,0.022746,0.159995,0.943380,-1.684952,-0.757742,0.133884,-0.993149,0.483346
5078,logo_9,5,0.343754,-2.241734,1.158007,0.410447,-0.030505,0.184853,-2.326611,-0.864749,...,-2.589544,-2.719802,0.475695,-0.015311,0.513772,-1.305443,-0.622814,0.806337,-1.464364,0.720238
5079,logo_9,6,0.689056,-1.528429,1.464514,-1.316596,-0.168186,-1.017659,-1.739284,-0.072727,...,-3.083637,-4.069217,0.112029,0.544818,0.756594,-1.739826,-0.292229,0.156785,0.124956,1.090774
5080,logo_9,8,0.685264,-1.755448,0.831120,1.118223,-0.383849,1.630368,-1.030001,-0.660419,...,-0.688228,-3.076010,-0.196317,0.816448,0.905165,-1.279724,-0.790978,0.470021,-0.793302,-0.150052


In [ ]:
#combined_embedding = combine_embeddings(df_svg_embedding=svgs_embedding, df_decomposed_svg_embedding=decomposed_svgs_embedding, model_path=model, save=False)

In [15]:
#file = "data/decomposed_svgs/L_Sonja_1_0.svg"
#file = "data/decomposed_svgs/L_Sonja_5_10.svg"
file = "data/decomposed_svgs/logo_0_0.svg"
z = encode_svg(filename = file, data_folder="data/decomposed_svgs")

In [17]:
#decode_z(z)

In [ ]:
svg = SVG.load_svg("data/svgs/L_7sins_04.svg")
svg.fill_(False).normalize().zoom(0.9)
svg = svg.canonicalize().simplify_heuristic()
svg.draw()

#### 5 Get Style Attributes
Get local as well as global style attributes of SVGs.

In [7]:
df_local = get_local_style_attributes("data/svgs")
df_global = get_global_style_attributes("data/svgs")

In [8]:
df_all_style_attributes = combine_style_attributes(df_global, df_local)
df_all_style_attributes

,file,animation_id,class_,fill,stroke,stroke_width,opacity,stroke_opacity
0,L_7sins_01.svg,0,st0,#FABB00,#000000,0,1.0,1.0
1,L_7sins_01.svg,1,st1,#C5007B,#000000,0,1.0,1.0
2,L_7sins_01.svg,2,st2,#EC7404,#000000,0,1.0,1.0
3,L_7sins_01.svg,3,st1,#C5007B,#000000,0,1.0,1.0
4,L_7sins_01.svg,4,st2,#EC7404,#000000,0,1.0,1.0
...,...,...,...,...,...,...,...,...
5095,L_Weidenbachhof_9_2.svg,22,st3,#FFFFFF,#000000,0,1.0,1.0
5096,L_Weidenbachhof_9_2.svg,23,st3,#FFFFFF,#000000,0,1.0,1.0
5097,L_Weidenbachhof_9_2.svg,24,st3,#FFFFFF,#000000,0,1.0,1.0
5098,L_Weidenbachhof_9_2.svg,25,st3,#FFFFFF,#000000,0,1.0,1.0


In [ ]:
#df_all_style_attributes.to_csv("data/style_attributes.csv")

In [ ]:
#output = open("data/style_attributes.pkl", 'wb')
#pickle.dump(df_all_style_attributes, output)
#output.close()

#### 6 Insert Animations

In [5]:
file = "data/svgs/L_7sins_02.svg"
animation_id = sort_by_relevance("data/path_selection/L_7sins_02")

In [6]:
for s in range(200):
    model_output = random_animation_vector(8, seed=s)
    create_animated_svg(file, animation_id, model_output, str(s))

In [5]:
with open('data/path_embedding.pkl', 'rb') as f:
    svgs_embedding = pickle.load(f)

#svgs_embedding[svgs_embedding['filename'] == f"{logo_name}"]

In [6]:
svgs_embedding#.dropna()

,filename,animation_id,0,1,2,3,4,5,6,7,...,246,247,248,249,250,251,252,253,254,255
0,L_7sins_01,0,0.528078,-0.821644,-0.122652,1.411909,-0.655937,1.278711,-0.606417,-1.533502,...,-0.229828,-0.810564,0.419082,-0.331659,0.091425,-1.093283,-1.394454,0.818913,-0.578239,-0.090547
1,L_7sins_01,1,1.274369,-2.215594,0.418359,0.322488,0.543223,0.864393,-0.773602,-1.031192,...,-0.856521,-2.083487,0.715938,1.431139,0.320074,-1.069192,-0.150538,0.768586,0.159383,-0.829308
8,L_7sins_01,3,1.379206,-1.854066,0.611372,0.672185,0.402664,1.554239,-0.366231,-0.733832,...,0.311397,-2.273414,0.038978,2.380411,0.803548,-0.861410,-0.408416,-0.068678,0.933381,-0.879638
9,L_7sins_01,2,0.472991,-1.226775,1.533987,-0.424805,-0.669460,1.127602,-0.713290,-0.084230,...,-1.234275,-2.134554,0.293492,1.386334,0.820009,-1.371945,-0.019476,-0.042061,0.339579,0.314696
10,L_7sins_01,4,1.223025,-1.810805,1.154082,1.085018,-0.081046,1.394158,-1.343777,-0.989737,...,-0.258803,-1.559884,-0.182104,0.341202,0.294003,-1.128353,-0.782518,-0.008047,-0.622554,-0.243537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4982,L_Weidenbachhof_9_2,5,1.420508,0.284751,1.306004,-0.239025,-0.600043,-1.303397,-0.727595,0.684894,...,0.050360,-3.857002,-0.576333,1.011679,0.867470,-1.261334,-1.125669,-1.123603,1.967078,1.598536
4983,L_Weidenbachhof_9_2,6,0.218143,-1.502262,1.006997,-0.218282,0.062004,0.051366,-2.292396,0.426296,...,-2.013161,-3.315885,-0.143858,1.519482,1.645396,-1.084207,-0.555623,-0.318483,-0.063448,1.268676
4984,L_Weidenbachhof_9_2,8,0.861285,-1.686104,1.262487,-1.128223,-0.187294,0.017669,-1.747269,-0.196984,...,-2.422992,-4.705516,0.285818,1.774886,1.593126,-1.800894,-0.586864,-0.000774,0.200038,0.794936
4985,L_Weidenbachhof_9_2,7,0.234749,-0.915931,1.484314,-0.272840,0.099517,-1.041811,-2.000403,0.361636,...,-1.541775,-3.713866,0.388559,1.408807,1.181244,-0.996578,-0.816932,-0.815901,0.835977,1.768944


In [20]:
svgs_embedding['filename'].apply(lambda row: row.split('_')[0:-1])

0                  [L, 7sins]
1                  [L, 7sins]
8                  [L, 7sins]
9                  [L, 7sins]
10                 [L, 7sins]
                ...          
4982    [L, Weidenbachhof, 9]
4983    [L, Weidenbachhof, 9]
4984    [L, Weidenbachhof, 9]
4985    [L, Weidenbachhof, 9]
4986    [L, Weidenbachhof, 9]
Name: filename, Length: 4974, dtype: object

In [26]:
svgs_embedding['filename'].apply(lambda row: "_".join(row.split('_')[0:-1]))

0                 L_7sins
1                 L_7sins
8                 L_7sins
9                 L_7sins
10                L_7sins
              ...        
4982    L_Weidenbachhof_9
4983    L_Weidenbachhof_9
4984    L_Weidenbachhof_9
4985    L_Weidenbachhof_9
4986    L_Weidenbachhof_9
Name: filename, Length: 4974, dtype: object

In [25]:
pd.set_option("display.max_rows", None)

In [7]:
from os import listdir
from os.path import isfile, join

In [20]:
decomposed_files = [f[:-4] for f in listdir('data/decomposed_svgs') if isfile(join('data/decomposed_svgs', f))]

In [21]:
filename_list = svgs_embedding['filename'].tolist()
animation_id_list = svgs_embedding['animation_id'].tolist()

In [22]:
filename_animation_list = [f'{filename_list[i]}_{animation_id_list[i]}' for i in range(len(filename_list))]

In [23]:
len(decomposed_files)

5109

In [24]:
len(filename_animation_list)

5095

In [30]:
set(decomposed_files)-set(filename_animation_list)

{'L_Braumann_01_126',
 'L_Braumann_02_134',
 'L_Braumann_05_134',
 'L_Braumann_06_135',
 'L_Braumann_07_134',
 'L_Braumannn_03_134',
 'L_Haarmonie_L_10_1',
 'L_Kinesiologie_01_21',
 'L_Kinesiologie_07_0',
 'L_Prime_People_5_1_19',
 'L_Sonja_1_0',
 'L_Sonja_2_0',
 'L_compassus_1_0',
 'L_compassus_2_0'}

In [30]:
os.getcwd()

'C:\\Users\\sarah\\PycharmProjects\\animate_logos'